### Imports

In [26]:
import pandas as pd
import mysql.connector
import os
import matplotlib.pyplot as plt
import plotly.express as px

### DB Connection

In [27]:
user = os.environ.get("DB_USER")
password = os.environ.get("DB_PASS")

In [28]:
connection = mysql.connector.connect(host = 'localhost',
                                    user = user,
                                    passwd = password,
                                    db = '2021_nfl')

### The Dataset

In [29]:
# the entire 2021 dataset  
rec2021 = pd.read_sql_query('''

SELECT 
    *
FROM
    `2021_wr_stats`
''', connection)

# the entire 2020 dataset  
rec2020 = pd.read_sql_query('''

SELECT 
    *
FROM
    `2020_wr_stats`
''', connection)
rec2020.head()

,player,player_id,position,team_name,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,contested_catch_rate,contested_receptions,...,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,week,dk_yards,dk_td,dj_bonus,dk_total,year
0,Davante Adams,8688,WR,GB,1,9.9,1,82.4,66.7,2,...,40,2.9,11.1,3.63,1,15.6,12,3,30.6,2020
1,DeAndre Hopkins,7808,WR,ARZ,1,6.3,3,87.5,100.0,1,...,68,4.9,10.8,3.08,1,15.1,0,3,18.1,2020
2,Amari Cooper,9437,WR,DAL,1,8.6,0,76.9,50.0,1,...,14,1.4,8.1,1.88,1,8.1,0,0,8.1,2020
3,Russell Gage,44771,WR,ATL,1,7.6,4,75.0,50.0,1,...,48,5.3,12.7,2.48,1,11.4,0,3,14.4,2020
4,Julio Jones,6158,WR,ATL,1,15.8,1,75.0,75.0,3,...,26,2.9,17.4,3.34,1,15.7,0,3,18.7,2020


In [30]:
# I am only interested in WRs right now so i will grab those
wr2021 = pd.read_sql_query('''

SELECT
    *
FROM
    `2021_wr_stats`
WHERE 
    position = 'WR'

''', connection)
wr2021.head()

,player,player_id,position,team_name,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,contested_catch_rate,contested_receptions,...,yards_after_catch_per_reception,yards_per_reception,yprr,week,dk_yards,dk_td,dk_reception,dk_bonus,dk_total,year
0,Amari Cooper,9437,WR,DAL,1,8.8,2,76.5,75.0,3,...,2.3,10.7,2.40,1,13.9,12,13,3,41.9,2021
1,Tyreek Hill,10799,WR,KC,1,15.8,0,73.3,0.0,0,...,4.5,17.9,5.18,1,19.7,6,11,3,39.7,2021
2,CeeDee Lamb,61570,WR,DAL,1,11.7,1,46.7,0.0,0,...,8.6,14.9,2.21,1,10.4,6,7,3,26.4,2021
3,Chris Godwin,11839,WR,TB,1,9.3,4,69.2,0.0,0,...,6.1,11.7,2.10,1,10.5,6,9,3,28.5,2021
4,Cole Beasley,7330,WR,BUF,1,5.9,1,66.7,0.0,0,...,3.3,7.5,1.09,1,6.0,0,8,0,14.0,2021


### Some EDA

In [31]:
# color codes for each team
team_colors = {
                "LA": '#003594',
                "MIN": '#4F2683',
                "GB": "#203731",
                "CIN": "#FB4F14",
                "SF": "#AA0000",
              "KC": "#E31837",
              "BUF": "#00338D",
              "SEA": "#69BE28",
              "PIT": "#FFB612",
              "CAR": "#0085CA",
              "LAC": "#0080C6",
              "TB": "#D50A0A",
              "DAL": "#041E42",
              "IND": "#002C5F",
              "CHI": "#C83803",
              "WAS": "#5A1414",
              "LV": "#000000",
              "HST": "#A71930",
              "MIA": "#008E97",
              "BLT": "#241773",
              "ARZ": "#97233F",
              "PHI": "#004C54",
              "DET": "#B0B7BC",
              "TEN": "#4B92DB",
              "NE": "#002244",
              "JAX": "#D7A22A",
              "DEN": "#FB4F14",
              "ATL": "#A71930",
              "NO": "#D3BC8D",
              "CLV": "#311D00",
              "NYJ": "#125740"}

In [32]:
# Lets take a look at the top 50 wr by yards
top50yards = pd.read_sql_query('''

SELECT
    player,
    team_name,
    SUM(yards) as 'total_yards',
    AVG(avg_depth_of_target) as adt,
    SUM(player_game_count),
    SUM(avoided_tackles) as broken_tackles
    
FROM
    `2021_wr_stats`
WHERE 
    position = 'WR'
GROUP BY player
ORDER BY total_yards DESC
LIMIT 50

''', connection)

px.bar(top50yards,
      x='player',
      y='total_yards',
      title='Top 50 WR Yards 2021',
       color='team_name',
      color_discrete_map=team_colors
       ,).update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

In [33]:
# Now lets see how avg depth of target effected overall yardage
px.scatter(top50yards,
           x='adt',
           y='total_yards',
           title='ADT by yards',
           color='team_name',
           color_discrete_map=team_colors,
           text='player').update_traces(textposition='top center')

In [34]:
# Now lets see how broken tackles effect total yards
px.scatter(top50yards,
           x='broken_tackles',
           y='total_yards',
           title='ADT by yards',
           color='team_name',
           color_discrete_map=team_colors,
           text='player').update_traces(textposition='top center')

In [35]:
# grabbing Justin Jefferson's career stats
jjStats = pd.read_sql_query('''

SELECT * from (SELECT 
	player,
    player_id,
    team_name,
    yards,
    week,
    year
FROM
	`2021_wr_stats` 
UNION
	SELECT 
	player,
    player_id,
    team_name,
    yards,
    week,
    year
FROM
	`2020_wr_stats` ) as jj
WHERE player = "Justin Jefferson"

''', connection)

# grabbing Justin Jefferson's 2021 stats
jj2021Stats = pd.read_sql_query('''

SELECT
    *
FROM 
    `2021_wr_stats`
WHERE
    player = 'Justin Jefferson'
''', connection)

jjStats.head()


,player,player_id,team_name,yards,week,year
0,Justin Jefferson,61398,MIN,71,1,2021
1,Justin Jefferson,61398,MIN,65,2,2021
2,Justin Jefferson,61398,MIN,118,3,2021
3,Justin Jefferson,61398,MIN,84,4,2021
4,Justin Jefferson,61398,MIN,124,5,2021


In [36]:
# graphing JJ's career yards per game
px.line(jjStats,
        x='week',
        y='yards',
        color='year',
        markers=True)

### Some Preseason Stuff

In [37]:
# gathering 2021 preseason data
preseasonQB = pd.read_sql_query('''

SELECT
    *
FROM
    preseasonw1

''', connection)
preseasonQB.head()

,player,player_id,position,team_name,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,avg_time_to_throw,...,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa,preseason_week,dk_yards,dk_touchdown,dk_bonus,dk_total
0,Nathan Peterman,11926,QB,LV,1,86.5,37,39,5.1,2.60,...,0,2,4.3,246,6.3,1,9.84,0,0,9.84
1,Kyle Lauletta,46452,QB,CLV,1,84.6,26,27,8.7,2.38,...,2,0,0.0,212,7.9,1,8.48,12,0,20.48
2,Chris Streveler,46440,QB,ARZ,1,61.9,21,24,10.0,2.57,...,0,0,0.0,107,4.5,1,4.28,0,0,4.28
3,Chase Daniel,5214,QB,LAC,1,72.7,22,24,7.9,2.32,...,0,0,0.0,104,4.3,1,4.16,0,0,4.16
4,Steven Montez,46465,QB,WAS,1,70.8,24,24,2.8,2.93,...,1,0,0.0,108,4.5,1,4.32,6,0,10.32


In [38]:
# join 2021 preseason with 2021 reg season stats through w3
qb_pre_to_one = pd.read_sql_query('''

SELECT
    wo.player,
    wo.team_name,
    sum(DISTINCT(ps.attempts)) as preseason_attempts,
    sum(DISTINCT(ps.yards)) as preseason_yards,
    sum(DISTINCT(ps.touchdowns)) as preseason_touchdowns,
    sum(DISTINCT(ps.turnover_worthy_plays)) as preseason_turnover_worthy_plays,
    (SUM(DISTINCT(ps.yards)) / SUM(DISTINCT(ps.attempts))) as preseason_ypa,
    sum(DISTINCT(wo.attempts)) as three_attempts,
    SUM(DISTINCT(wo.yards)) as week_one_yards,
    SUM(DISTINCT(wo.touchdowns)) as week_one_toucdowns,
    AVG(DISTINCT(wo.grades_pass)) as passing_grade
FROM
    preseasonw1 ps
INNER JOIN
    `qb_reg_season` wo
ON
    wo.player = ps.player
GROUP BY 
    ps.player
    HAVING SUM(wo.attempts) >= 30


''', connection)
qb_pre_to_one.head()

,player,team_name,preseason_attempts,preseason_yards,preseason_touchdowns,preseason_turnover_worthy_plays,preseason_ypa,three_attempts,week_one_yards,week_one_toucdowns,passing_grade
0,Andy Dalton,CHI,21.0,164.0,1.0,0.0,7.8095,49.0,262.0,1.0,72.800000
1,Baker Mayfield,CLV,10.0,113.0,1.0,0.0,11.3000,80.0,780.0,1.0,74.033333
2,Ben Roethlisberger,PIT,10.0,137.0,2.0,0.0,13.7000,130.0,801.0,1.0,55.633333
3,Daniel Jones,NYG,22.0,135.0,1.0,2.0,6.1364,104.0,782.0,1.0,76.566667
4,Davis Mills,HST,65.0,333.0,2.0,5.0,5.1231,46.0,270.0,1.0,48.000000


In [39]:
# snap counts for each qb
qb_snaps = pd.read_sql_query('''

SELECT
    player,
    team_name,
    SUM(attempts) as preseason_attempts,
    sum(yards) as preseason_yards,
    sum(touchdowns) as preseason_touchdowns,
    sum(turnover_worthy_plays) as preseason_turnover_worthy_plays,
    (yards / attempts) as preseason_ypa
FROM
    preseasonw1 ps
GROUP BY 
    ps.player
''', connection)
qb_snaps.head()

,player,team_name,preseason_attempts,preseason_yards,preseason_touchdowns,preseason_turnover_worthy_plays,preseason_ypa
0,Nathan Peterman,LV,92.0,593.0,2.0,4.0,6.3077
1,Kyle Lauletta,CLV,72.0,514.0,3.0,3.0,7.8519
2,Chris Streveler,ARZ,30.0,181.0,1.0,0.0,4.4583
3,Chase Daniel,LAC,57.0,234.0,0.0,3.0,4.3333
4,Steven Montez,WAS,34.0,136.0,1.0,0.0,4.5000


In [40]:
# lets see how preseason attempts effects average pff grade through 3 weeks
preseason_to_week_one = px.scatter(qb_pre_to_one,
                                   x='passing_grade',
                                   y='preseason_attempts',
                                   labels=dict(passing_grade="Average passing grade", preseason_attempts="Total preseason passing attempts"),
                                   text='player',
                                   color='team_name',
                                   color_discrete_map=team_colors,).update_traces(textposition='top center')
preseason_to_week_one.update_layout(showlegend=False)
preseason_to_week_one.update_layout(
    title={
        'text': "AVG PFF passing grade through 3 weeks vs preseason attempts (2021) <br><sup>Minimum 30 attempts through week 3</sup>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
preseason_to_week_one.show()

In [41]:
# qbs that didnt play in preseason
qb_no_preseason = pd.read_sql_query('''


SELECT
    wo.player,
    AVG(wo.grades_pass) as passing_grade,
    sum(wo.attempts) as three_attempts,
    ps.attempts as preseason_attempts
FROM
	`qb_reg_season` wo
    LEFT JOIN
		`preseasonw1`ps
	ON
		ps.player = wo.player
GROUP BY 
    wo.player
HAVING
	SUM(wo.attempts) > 10 and ps.attempts IS NULL
    
    ''', connection)

qb_no_preseason.head()

,player,passing_grade,three_attempts,preseason_attempts
0,Dak Prescott,73.433333,111.0,None
1,Derek Carr,80.366667,136.0,None
2,Justin Herbert,76.833333,126.0,None
3,Carson Wentz,65.900000,106.0,None
4,Matt Ryan,59.300000,117.0,None


In [42]:
# WRs preseason 2022
presea_wr_2022 = pd.read_sql_query('''
SELECT 
    * 
FROM
    `wr_preseason_2022`
WHERE
    position = "WR"

''', connection)
presea_wr_2022.head(3)

,player,player_id,position,depth,team_name,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,contested_catch_rate,...,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,dk_yards,dk_touchdowns,dk_bonus,dk_total
0,Kristian Wilkerson,28729,WR,3,NE,1,11.7,3,66.7,50,...,32,99,23,2.9,12.4,3.09,9.9,0,0,9.9
1,Tom Kennedy,94379,WR,2,DET,1,16.1,0,72.7,100,...,22,104,21,2.6,13.0,3.25,10.4,0,3,13.4
2,Tim Jones,61669,WR,2,JAX,2,11.0,0,66.7,100,...,3,46,9,1.5,7.7,0.85,4.6,0,0,4.6


In [43]:
# Lets see if there is any correlation between roster position and total draft kings points
px.scatter(presea_wr_2022,
          x='dk_total',
          y='depth')

In [44]:
preWRTotal = presea_wr_2022[['player', 'depth', 'team_name', 'dk_total']]
preWRTotal.head()

,player,depth,team_name,dk_total
0,Kristian Wilkerson,3,NE,9.9
1,Tom Kennedy,2,DET,13.4
2,Tim Jones,2,JAX,4.6
3,Makai Polk,3,BLT,4.3
4,Lil'Jordan Humphrey,4,NE,12.2


In [45]:
# lets get the total dk points by depth position for each team
pd.set_option('display.max_rows', None)
print(preWRTotal.groupby(['team_name', 'depth'])['dk_total'].sum())

team_name  depth
ARZ        2         5.3
           3        12.9
           4         8.0
           5         0.0
ATL        1         2.4
           2         5.2
           3         2.7
           4         0.0
BLT        2         0.3
           3        12.2
           4        13.0
BUF        2        10.0
           3        10.3
           4         1.0
CAR        1         0.0
           2         8.2
           3         6.6
           4         3.8
           5         4.6
           6         0.0
CHI        1         2.6
           3         2.5
           4        16.1
CIN        2         7.4
           3        19.2
CLV        2         0.7
           3         0.5
           4         2.5
           5         0.0
DAL        1         1.0
           2         1.8
           3        13.0
           4         2.4
DEN        2        12.1
           3        15.5
           4         9.4
           5         1.1
DET        1         2.9
           2        18.6
        

In [46]:
# Lets take a look to see how rookies performed in week one
# also interested in seeing how draft position is effecting pff grade
rushing_rookie_one = pd.read_sql_query('''

SELECT 
    *
FROM
    `preseason_rushing_rookies_1`

''', connection)

tackle_rookie_one = pd.read_sql_query('''

SELECT 
    *
FROM
    `preseason_blocking_rookies_1`
WHERE 
    position = "T"

''', connection)

guard_rookie_one = pd.read_sql_query('''

SELECT 
    *
FROM
    `preseason_blocking_rookies_1`
WHERE 
    position = "G"

''', connection)

center_rookie_one = pd.read_sql_query('''

SELECT 
    *
FROM
    `preseason_blocking_rookies_1`
WHERE 
    position = "C"

''', connection)


center_rookie_one.head(2)

,player,player_id,position,team_name,player_game_count,draft_round,draft_pick,block_percent,declined_penalties,franchise_id,...,snap_counts_ce,snap_counts_lg,snap_counts_lt,snap_counts_offense,snap_counts_pass_block,snap_counts_pass_play,snap_counts_rg,snap_counts_rt,snap_counts_run_block,snap_counts_te
0,Luke Wattenberg,41024,C,DEN,1,5,171,100,0,10,...,14,0,0,51,38,38,37,0,13,0
1,Doug Kramer,41458,C,CHI,1,6,207,100,0,6,...,48,0,0,48,25,25,0,0,23,0


In [53]:
# chart of draft pick by pff grade
pre_week_one_rushing_rookies = px.scatter(rushing_rookie_one,
          x='grades_offense',
          y='draft_pick',
          text='player',
          labels=dict(grades_offense="PFF offense grade", draft_pick="Draft position"),                                
          color='team_name',
          color_discrete_map=team_colors,
          title="Preseason Game 1 (2022) PFF offensive grade vs draft position for rookie RBs").update_traces(textposition='top center')
def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]
pre_week_one_rushing_rookies.update_traces(textposition=improve_text_position(rushing_rookie_one['grades_offense']))
pre_week_one_rushing_rookies.update_traces(textposition='top center')
pre_week_one_rushing_rookies.update_layout(showlegend=False)
pre_week_one_rushing_rookies.show()

In [48]:
pre_week_one_tackle_rookies = px.scatter(tackle_rookie_one,
          x='grades_offense',
          y='draft_pick',
          text='player',
          labels=dict(grades_offense="PFF offense grade", draft_pick="Draft position"),                                
          color='team_name',
          color_discrete_map=team_colors,
          title="Preseason Game 1 (2022) PFF offensive grade vs draft position for rookie offensive tackles").update_traces(textposition='top center')
pre_week_one_tackle_rookies.update_traces(textposition='top center')
pre_week_one_tackle_rookies.update_layout(showlegend=False)
pre_week_one_tackle_rookies.show()

In [49]:
pre_week_one_guard_rookies = px.scatter(guard_rookie_one,
          x='grades_offense',
          y='draft_pick',
          text='player',
          labels=dict(grades_offense="PFF offense grade", draft_pick="Draft position"),                                
          color='team_name',
          color_discrete_map=team_colors,
          title="Preseason Game 1 (2022) PFF offensive grade vs draft position for rookie offensive guards").update_traces(textposition='top center')
pre_week_one_guard_rookies.update_traces(textposition='top center')
pre_week_one_guard_rookies.update_layout(showlegend=False)
pre_week_one_guard_rookies.show()

In [50]:
pre_week_one_center_rookies = px.scatter(center_rookie_one,
          x='grades_offense',
          y='draft_pick',
          text='player',
          labels=dict(grades_offense="PFF offense grade", draft_pick="Draft position"),                                
          color='team_name',
          color_discrete_map=team_colors,
          title="Preseason Game 1 (2022) PFF offensive grade vs draft position for rookie centers").update_traces(textposition='top center')
pre_week_one_center_rookies.update_traces(textposition='top center')
pre_week_one_center_rookies.update_layout(showlegend=False)
pre_week_one_center_rookies.show()